# Asset Pricing Assignment: Question 3

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import statsmodels.api as sm
from scipy.stats import f
from scipy.stats import t

3a. Theory-based question.

3b. Check whether there is any abnormal return variation across portfolios of stocks with different investment growth rates. Estimate the CAPM for each of the 10 portfolios and plot the CAPM $\alpha$'s against investment deciles.

In [ ]:
# read file F-F_Research_Data_Factors.CSV
dfff = pd.read_csv('F-F_Research_Data_Factors.CSV', nrows=733, sep=',')

# rename column 0 as 'Date'
dfff.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

# read Date as integer
dfff['Date'] = dfff['Date'].astype(int)

dfff.head()

In [ ]:
# read file Portfolios_Formed_on_INV_Cleaned_MktR.CSV
dfinv = pd.read_csv('Portfolios_Formed_on_INV_Cleaned_MktR.csv', nrows=734, skiprows=1, sep=',')

# rename column 0 as 'Date'
dfinv.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

# read Date as integer
dfinv['Date'] = dfinv['Date'].astype(int)

print('Raw Returns:')
print(dfinv.head())

print('Excess Returns:')

# get excess returns of 10 decile portfolios
for column in dfinv.columns[1:-1]:
    dfinv[column] = dfinv[column] - dfff['RF']

dfinv.head()

In [ ]:
# estimate the CAPM for 10 portfolios
CAPM_results = {}

# assume homoskedasticity
for model in dfinv.columns[9:19]:
    x = dfinv[['Mkt-RF']]
    x = sm.add_constant(x)
    CAPM_results[model] = sm.OLS(dfinv[model], x).fit()

CAPM_results

In [ ]:
# extract alphas from regression results
alpha_list = []

for key in CAPM_results:
    a = CAPM_results[key]
    alpha_list.append({"Portfolio": key, "Alpha": round(a.params['const'], 2)})

alphas = pd.DataFrame(alpha_list)
display(alphas)

In [ ]:
# plot CAPM alphas against investment deciles
deciles_list = []
x = range(10, 101, 10)
for n in x:
    deciles_list.append(n)

deciles = pd.DataFrame(deciles_list)
alpha_plot = pd.concat([deciles, alphas],axis=1,ignore_index=True)
alpha_plot = alpha_plot.rename(columns={0: "Decile", 1: "Portfolio", 2: "Alpha"})
alpha_plot.drop(['Portfolio'],axis=1,inplace=True)

alpha_plot

In [ ]:
# plot CAPM alphas against investment deciles
plt.figure(figsize=(8, 5))
plt.plot(alpha_plot['Decile'], alpha_plot['Alpha'], marker='o', linestyle='-')

plt.xlabel('Investment Decile')
plt.ylabel('CAPM Alpha')
plt.title('CAPM Alpha against Investment Decile')
plt.grid(True)
plt.show()

3c. Theory-based question.

3d. Consider the classic Fama-French model and its extension that includes the investment growth factor. Estimate the two models given by eq. (1) and (2) on the 25 portfolios used in Question (1). For each model, report in a table the adjusted R2, α and its corresponding t-statistics. Then, again for each model, report the GRS test statistics and the corresponding p-value. Compare your results for the two models.

In [ ]:
# create INV (investment growth factor)
dfinv["INV"] = (dfinv["Hi 30"] - dfinv["Lo 30"])

dfinv.head()

In [ ]:
# create extended Fama-French Factors matrix
dfff_extended = dfff[['Date','SMB','HML','RF']]

# add INV column
dfff_extended = pd.merge(dfff_extended, dfinv, on="Date")
dfff_extended = dfff_extended[["Date","Mkt-RF", "SMB", "HML", "INV","RF"]]

dfff.head()

In [ ]:
# read file 25_Portfolios_Cleaned.csv
df25p = pd.read_csv('25_Portfolios_Cleaned.csv', nrows=733, sep=',')

# rename column 0 as 'Date'
df25p.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

# read Date as integer
df25p['Date'] = df25p['Date'].astype(int)

print('Raw Returns:')
print(df25p.head())

print('Excess Returns:')

# get excess returns for 25 portfolios
for column in df25p.columns[1:]:
    df25p[column] = df25p[column] - dfff_extended['RF']

df25p.head()

In [ ]:
# regression for classic 3-factor Fama-French model
FF_classic = {}

# assume homoskedasticity
for portfolio in df25p.columns[1:]:
    x = dfff_extended[['Mkt-RF','SMB','HML']]
    x = sm.add_constant(x)
    FF_classic[portfolio] = sm.OLS(df25p[portfolio], x).fit()

FF_classic

In [ ]:
# regression for Fama-French model with INV
FF_extended = {}

# assume homoskedasticity
for portfolio in df25p.columns[1:]:
    x = dfff_extended[['Mkt-RF','SMB','HML','INV']]
    x = sm.add_constant(x)
    FF_extended[portfolio] = sm.OLS(df25p[portfolio], x).fit()

FF_extended

In [ ]:
# report adjusted R-squared, alphas, t-statistics for classic Fama-French model
ffclassic_r2_dict = {}
ffclassic_alpha_dict = {}
ffclassict_const_dict = {}
#ffclassict_mktrf_dict = {}
#ffclassict_smb_dict = {}
#ffclassict_hml_dict = {}

for key in FF_classic:
    res = FF_classic[key]
    ffclassic_r2_dict[key] = round(res.rsquared_adj,5)
    ffclassic_alpha_dict[key] = round(res.params['const'], 3)
    ffclassict_const_dict[key] = round(res.tvalues['const'], 2)
    #ffclassict_mktrf_dict[key] = round(res.tvalues['Mkt-RF'], 2)
    #ffclassict_smb_dict[key] = round(res.tvalues['SMB'], 2)
    #ffclassict_hml_dict[key] = round(res.tvalues['HML'], 2)

ffclassic = pd.DataFrame({"Adjusted R-squared": ffclassic_r2_dict, "Alpha": ffclassic_alpha_dict, "t-statistic (Alpha)": ffclassict_const_dict})
ffclassic

In [ ]:
# report adjusted R-squared, alphas, t-statistics for extended Fama-French model with INV regressor
ffextended_r2_dict = {}
ffextended_alpha_dict = {}
ffextendedt_const_dict = {}
#ffextendedt_mktrf_dict = {}
#ffextendedt_smb_dict = {}
#ffextendedt_hml_dict = {}
#ffextendedt_inv_dict = {}

for key in FF_extended:
    res = FF_extended[key]
    ffextended_r2_dict[key] = round(res.rsquared_adj,5)
    ffextended_alpha_dict[key] = round(res.params['const'], 3)
    ffextendedt_const_dict[key] = round(res.tvalues['const'], 2)
    #ffextendedt_mktrf_dict[key] = round(res.tvalues['Mkt-RF'], 2)
    #ffextendedt_smb_dict[key] = round(res.tvalues['SMB'], 2)
    #ffextendedt_hml_dict[key] = round(res.tvalues['HML'], 2)
    #ffextendedt_inv_dict[key] = round(res.tvalues['INV'], 2)

ffextended = pd.DataFrame({"Adjusted R-squared": ffextended_r2_dict, "Alpha": ffextended_alpha_dict, "t-statistic (Alpha)": ffextendedt_const_dict})
ffextended

In [ ]:
# adjusted R^2, alphas, t-statistics for both models
dfallstats = pd.DataFrame({"Classic Model Adjusted R-squared": ffclassic_r2_dict, "Classic Model Alpha": ffclassic_alpha_dict, "Classic Model t-statistic (Alpha)": ffclassict_const_dict, "Extended Model Adjusted R-squared": ffextended_r2_dict, "Extended Model Alpha": ffextended_alpha_dict, "Extended Model t-statistic (Alpha)": ffextendedt_const_dict})

dfallstats

In [ ]:
significant_counts = (dfallstats['Classic Model t-statistic (Alpha)'].abs() > 1.96).sum()
significant_counts

In [ ]:
# GRS test for both models 

# Fama-French classic model

# convert alphas into DataFrame
alphahat_classic = pd.DataFrame([ffclassic_alpha_dict])

# create var-cov matrix for residuals and invert (25x25)
ffclassic_resid =np.zeros((df25p.shape[0],25))
i=0
for key in FF_classic:
    res = FF_classic[key]
    ffclassic_resid[:,i] = res.resid
    i = i+1

sigmahat_classic = np.cov(ffclassic_resid, rowvar=False, bias=True)

sigmahat_classic_inv = pd.DataFrame(np.linalg.inv(sigmahat_classic))

# create omega hat var-cov matrix for factors and invert (3x3)
omegahat_classic_numpy = dfff_extended[["Mkt-RF", "SMB", "HML"]].to_numpy()

# not corrected for bias
# omegahat_classic = np.cov(omegahat_classic_numpy, rowvar=False, bias=True)

# corrected for bias
omegahat_classic = np.cov(omegahat_classic_numpy, rowvar=False)

omegahat_classic_inv = pd.DataFrame(np.linalg.inv(omegahat_classic))

# create ET(f) column vector
mean_mktrf = dfff_extended["Mkt-RF"].mean()
mean_smb = dfff_extended["SMB"].mean()
mean_hml = dfff_extended["HML"].mean()

etf_classic = pd.DataFrame([mean_mktrf,mean_smb,mean_hml])

# Fama-French extended model

# convert alphas into DataFrame
alphahat_extended = pd.DataFrame([ffextended_alpha_dict])

# create var-cov matrix for residuals and invert (25x25)
ffextended_resid =np.zeros((df25p.shape[0],25))
i=0
for key in FF_extended:
    res = FF_extended[key]
    ffextended_resid[:,i] = res.resid
    i = i+1

sigmahat_extended = np.cov(ffextended_resid, rowvar=False, bias=True)

sigmahat_extended_inv = pd.DataFrame(np.linalg.inv(sigmahat_extended))

# create omega hat variance matrix and invert (4x4)
omegahat_extended_numpy = dfff_extended[["Mkt-RF", "SMB", "HML","INV"]].to_numpy()

# not corrected for bias
# omegahat_extended = np.cov(omegahat_extended_numpy, rowvar=False, bias=True)

#corrected for bias
omegahat_extended = np.cov(omegahat_extended_numpy, rowvar=False)

omegahat_extended_inv = pd.DataFrame(np.linalg.inv(omegahat_extended))

# create ET(f) column vector
mean_mktrf = dfff_extended["Mkt-RF"].mean()
mean_smb = dfff_extended["SMB"].mean()
mean_hml = dfff_extended["HML"].mean()
mean_inv = dfff_extended["INV"].mean()

etf_extended = pd.DataFrame([mean_mktrf,mean_smb,mean_hml,mean_inv])


# define argument for GRS test stat
# N: number of portfolios, K: number of regressors, T: number of periods
# .values strips index to avoid matrices not aligned
def grs_test(N, K, T, alphahat, sigmahatinv, etf, omegahatinv):
    grs_stat = ((T-N-K)/N) * (alphahat.values @ sigmahatinv.values @ alphahat.T.values) / (1 + etf.T @ omegahatinv @ etf)
    grs_p_value = 1 - f.cdf(grs_stat, N, T-N-K)
    print(round(grs_stat,5))
    print(round(grs_p_value.item(),11))
    

# Fama-French classic model GRS test
print("F-F Classic Model:")
grs_test(25, 3, df25p.shape[0], alphahat_classic, sigmahat_classic_inv, etf_classic, omegahat_classic_inv)

# Fama-French extended model GRS test
print("F-F Extended Model:")
grs_test(25, 4, df25p.shape[0], alphahat_extended, sigmahat_extended_inv, etf_extended, omegahat_extended_inv)

In [ ]:
# compare mean, variance of 4 F-F model factors

for factor in dfff_extended:
    print(str(factor))
    print(dfff_extended[str(factor)].mean())
    print(round(dfff_extended[str(factor)].var(),2))

In [ ]:
df25p.shape

In [ ]:
# betas of INV factor
inv_betas_dict = {}
inv_betas_se = {}
inv_betas_t_manual = {}
inv_betas_t_auto = {}
inv_betas_p = {}
df_t = df25p.shape[0] - 5

for key in FF_extended:
    res = FF_extended[key]
    inv_betas_dict[key] = round(res.params['INV'],5)
    inv_betas_se[key] = res.bse['INV']
    inv_betas_t_manual[key] = res.params['INV']/res.bse['INV']
    inv_betas_t_auto[key] = round(res.tvalues['INV'], 5)
    inv_betas_p[key] = t.cdf(res.tvalues['INV'], df=df_t)


inv_betas = pd.DataFrame({"INV Betas": inv_betas_dict, "t-statistic (manual)": inv_betas_t_manual, "t-statistic (auto)": inv_betas_t_auto, "p-value": inv_betas_p})


inv_betas